# Geodistance

Veremos en esta notebook como calcular la distancia entre dos coordenadas. Usaremos geopy! (y veremos una tabla que armamos también)

In [14]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import geopy
import pandas as pd

dist_df = pd.read_csv("../data/geoloc/distancias.csv", index_col=0)

dist_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,buenosaires,cordoba,corrientes,formosa,laplata,larioja,mendoza,neuquen,entrerios,misiones,...,tucuman,jujuy,salta,sanjuan,sanluis,santafe,lapampa,santiago,tierradelfuego,rionegro
buenosaires,0,646,792,933,53,986,985,989,375,834,...,1080,1334,1282,1005,749,393,579,939,2373,799
cordoba,646,0,677,824,698,340,466,907,348,919,...,517,809,745,412,293,330,577,401,2618,1047
corrientes,792,677,0,157,830,814,1131,1534,500,291,...,633,742,719,1039,969,498,1136,535,3131,1527
formosa,933,824,157,0,968,927,1269,1690,656,263,...,703,750,741,1169,1117,654,1293,629,3284,1681
laplata,53,698,830,968,0,1038,1029,1005,427,857,...,1132,1385,1333,1053,795,444,602,991,2350,789
larioja,986,340,814,927,1038,0,427,1063,659,1098,...,330,600,533,283,435,640,834,311,2821,1311
mendoza,985,466,1131,1269,1029,427,0,676,790,1384,...,756,1023,957,152,235,775,586,713,2435,1019
neuquen,989,907,1534,1690,1005,1063,676,0,1053,1709,...,1370,1658,1591,824,643,1049,422,1286,1762,479
entrerios,375,348,500,656,427,659,790,1053,0,658,...,707,959,906,757,574,19,642,566,2635,1030
misiones,834,919,291,263,857,1098,1384,1709,658,0,...,924,1007,992,1306,1200,664,1293,827,3207,1624


¿es simétrica?

In [17]:
import numpy as np

np.array_equal(dist_df.values, dist_df.values.T)

True

In [41]:
import json

with open("../data/geoloc/argentina.json") as f:
    argentina_info = json.load(f) 
    
# Convertimos a float las coordenadas
for prov, info in argentina_info.items():
    coords = info["coord"].split(",")
    
    info["coord"] = (float(coords[0]), float(coords[1]))

In [42]:
from geopy import distance

distance.distance?

Init signature: distance.distance(*args, **kwargs)
Docstring:     
Calculate the geodesic distance between two points.

Set which ellipsoidal model of the earth to use by specifying an
``ellipsoid`` keyword argument. The default is 'WGS-84', which is the
most globally accurate model.  If ``ellipsoid`` is a string, it is
looked up in the `ELLIPSOIDS` dictionary to obtain the major and minor
semiaxes and the flattening. Otherwise, it should be a tuple with those
values.  See the comments above the `ELLIPSOIDS` dictionary for
more information.

Example::

    >>> from geopy.distance import geodesic
    >>> newport_ri = (41.49008, -71.312796)
    >>> cleveland_oh = (41.499498, -81.695391)
    >>> print(geodesic(newport_ri, cleveland_oh).miles)
    538.390445368


.. versionadded:: 1.13.0
File:           ~/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/geopy/distance.py
Type:           type


In [55]:
provincias = list(argentina_info.keys())

for i, prov1 in enumerate(provincias):
    for j in range(i, len(provincias)):
        prov2 = provincias[j]
        pl = 10
        precalculada = dist_df[prov1][prov2]
        coords1 = argentina_info[prov1]["coord"]
        coords2 = argentina_info[prov2]["coord"]
        
        geodes = geopy.distance.distance(coords1, coords2).km
        dif = abs(geodes - precalculada)
        
        if dif > 10:
            print("{:<10} - {:<10} --- {:<5.2f} {:<5.2f} --- {}".format(
                prov1[:pl], prov2[:pl], precalculada, geodes, dif))

misiones   - santafe    --- 664.00 653.42 --- 10.577230065968365


Sólo una distancia tiene problemas!